# Orchestrator Training Dataset Maker (ver. 1.0)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

#### Project Los Angeles

#### Tegridy Code 2022

***

# (SETUP ENVIRONMENT)

In [ ]:
#@title Install all dependencies (run only once per session)

!git clone https://github.com/asigalov61/Orchestrator
!pip install tqdm

In [ ]:
#@title Import all needed modules

print('Loading needed modules. Please wait...')
import os

import math
import statistics
import random

from tqdm import tqdm

if not os.path.exists('/content/Dataset'):
    os.makedirs('/content/Dataset')

print('Loading TMIDIX module...')
os.chdir('/content/Orchestrator')

import TMIDIX

print('Done!')

os.chdir('/content/')
print('Enjoy! :)')

# (DOWNLOAD SOURCE MIDI DATASET)

In [ ]:
#@title Download original LAKH MIDI Dataset

%cd /content/Dataset/

!wget 'http://hog.ee.columbia.edu/craffel/lmd/lmd_full.tar.gz'
!tar -xvf 'lmd_full.tar.gz'
!rm 'lmd_full.tar.gz'

%cd /content/

# (FILE LIST)

In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Save file list
###########

print('Loading MIDI files...')
print('This may take a while on a large dataset in particular.')

dataset_addr = "/content/Dataset"
# os.chdir(dataset_addr)
filez = list()
for (dirpath, dirnames, filenames) in os.walk(dataset_addr):
    filez += [os.path.join(dirpath, file) for file in filenames]
print('=' * 70)

if filez == []:
    print('Could not find any MIDI files. Please check Dataset dir...')
    print('=' * 70)

print('Randomizing file list...')
random.shuffle(filez)

TMIDIX.Tegridy_Any_Pickle_File_Writer(filez, '/content/drive/MyDrive/filez')

In [ ]:
#@title Load file list
filez = TMIDIX.Tegridy_Any_Pickle_File_Reader('/content/drive/MyDrive/filez')

# (PROCESS)

In [ ]:
#@title Process MIDIs with TMIDIX MIDI processor

print('=' * 70)
print('TMIDIX MIDI Processor')
print('=' * 70)
print('Starting up...')
print('=' * 70)

###########

START_FILE_NUMBER = 0
LAST_SAVED_BATCH_COUNT = 0
TRANSPOSE_PITCHES_TO_MEAN_C = False

input_files_count = START_FILE_NUMBER
files_count = LAST_SAVED_BATCH_COUNT

melody_chords_f = []
stats = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

print('Processing MIDI files. Please wait...')
print('=' * 70)

for f in tqdm(filez[START_FILE_NUMBER:]):
    try:
        input_files_count += 1

        fn = os.path.basename(f)
        fn1 = fn.split('.')[0]

        # Filtering out giant MIDIs
        file_size = os.path.getsize(f)

        if file_size < 200000:

          #=======================================================
          # START PROCESSING

          # Convering MIDI to ms score with MIDI.py module
          score = TMIDIX.midi2ms_score(open(f, 'rb').read())

          # INSTRUMENTS CONVERSION CYCLE
          events_matrix = []
          itrack = 1
          patches = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

          patch_map = [[0, 1, 2, 3, 4, 5, 6, 7], # Piano 
                      [24, 25, 26, 27, 28, 29, 30], # Guitar
                      [32, 33, 34, 35, 36, 37, 38, 39], # Bass
                      [40, 41], # Violin
                      [42, 43], # Cello
                      [46], # Harp
                      [56, 57, 58, 59, 60], # Trumpet
                      [71, 72], # Clarinet
                      [73, 74, 75], # Flute
                      [-1], # Drums
                      [52, 53], # Choir
                      [16, 17, 18, 19, 20] # Organ
                      ]

          while itrack < len(score):
              for event in score[itrack]:         
                  if event[0] == 'note' or event[0] == 'patch_change':
                      events_matrix.append(event)
              itrack += 1

          events_matrix.sort(key=lambda x: x[1])

          events_matrix1 = []

          for event in events_matrix:
                  if event[0] == 'patch_change':
                      patches[event[2]] = event[3]

                  if event[0] == 'note':
                      event.extend([patches[event[3]]])
                      once = False
                      
                      for p in patch_map:
                          if event[6] in p and event[3] != 9: # Except the drums
                              event[3] = patch_map.index(p)
                              once = True
                              
                      if not once and event[3] != 9: # Except the drums
                          event[3] = 15 # All other instruments/patches channel
                          event[5] = max(80, event[5])
                          
                      if event[3] < 12: # We won't write chans 12-16 for now...
                          events_matrix1.append(event)
                          stats[event[3]] += 1

          #=======================================================
          # PRE-PROCESSING

          # checking number of instruments in a composition
          instruments_list = list(set([y[3] for y in events_matrix1]))
          num_instr = len(instruments_list)

          # filtering out empty compositions and checking desired number of instruments in a composition
          # It had been observed that music models learn best from multi-instrumental music, even for solo instruments
          # So you can setup filtering by number of instruments here if you want

          if len(events_matrix1) > 0 and num_instr > 0:

            # recalculating timings
            for e in events_matrix1:
                e[1] = math.ceil(e[1] / 8) # Max 1 seconds for start-times
                e[2] = math.ceil(e[2] / 16) # Max 2 seconds for durations

            # Sorting by pitch, then by start-time
            events_matrix1.sort(key=lambda x: x[4], reverse=True)
            events_matrix1.sort(key=lambda x: x[1])

            # pitches augment stuff (calculating transpose value to C without drums)
            pitches = [y[4] for y in events_matrix1 if y[3] != 9]

            if len(pitches) > 0:
              avg_ptc = round(statistics.mean(pitches))
            else:
              avg_ptc = 0

            ptc_delta = 12 - (avg_ptc % 12)

            #=======================================================
            # FINAL PRE-PROCESSING

            first_event = True
            melody_chords = []
            pe = events_matrix1[0]
            pt = -1
            
            for e in events_matrix1:

                # Cliping all values...
                time = max(0, min(127, e[1]-pe[1]))
                dur = max(1, min(127, e[2]))
                cha = max(0, min(11, e[3]))
                ptc = max(1, min(127, e[4]))
                vel = max(8, min(127, e[5]))

                velocity = round(vel / 15)

                # Pitches shifting
                if cha != 9:
                  if TRANSPOSE_PITCHES_TO_MEAN_C:
                    ptc_aug = ptc + ptc_delta # Transposing composition to median C
                  else:
                    ptc_aug = ptc
                else:
                  ptc_aug = ptc + 128 # Shifting drums pitches because drums structure is different from non-drums

                # Time shifting
                if time == 0 and first_event == True:
                  abs_time = 0

                if time == 0 and first_event == False:
                  if pt == 0:
                    pass
                  else:
                    abs_time += 128

                if time !=0:
                  abs_time = time

                # Writing final note 
                melody_chords.append([abs_time, dur, cha, ptc_aug, velocity])

                pe = e
                pt = time
                
                first_event = False

            #=======================================================
            # FINAL PROCESSING
            #=======================================================

            # Break between compositions / Intro seq

            # 758 == SOS/EOS token
            # 759 == SOS/EOS token
            # 760-761 == Composition is without drums or with drums
            # 762-773 == Number of instruments in a composition

            # TOTAL DICTIONARY SIZE OF 774 TOKENS

            if 9 in instruments_list:
              drums_present = 761 # Yes
            else:
              drums_present = 760 # No
            
            melody_chords_f.extend([758, 759, drums_present, 762+(num_instr-1)])

            #=======================================================

            # Composition control seq
            intro_mode_time = statistics.mode([y[0] for y in melody_chords if y[2] != 9])
            intro_mode_dur = statistics.mode([y[1] for y in melody_chords if y[2] != 9])
            intro_mode_pitch = statistics.mode([y[3] for y in melody_chords if y[2] != 9])
            intro_mode_velocity = statistics.mode([y[4] for y in melody_chords if y[2] != 9])

            # Instrument value 12 is reserved for composition control seq
            intro_vel = (12 * 9) + intro_mode_velocity

            melody_chords_f.extend([intro_mode_time, intro_mode_dur+256, intro_mode_pitch+384, intro_vel+640])
            
            #=======================================================
            # MAIN PROCESSING CYCLE
            #=======================================================

            for m in melody_chords:

              # WRITING EACH NOTE HERE
              chan_vel = (m[2] * 9) + m[4]
              melody_chords_f.extend([m[0], m[1]+256, m[3]+384, chan_vel+640])

            #=======================================================
            
            # Processed files counter
            files_count += 1

            # Saving every 5000 processed files
            if files_count % 5000 == 0:
              print('SAVING !!!')
              print('=' * 70)
              print('Saving processed files...')
              print('=' * 70)
              print('Data check:', min(melody_chords_f), '===', max(melody_chords_f), '===', len(list(set(melody_chords_f))), '===', len(melody_chords_f))
              print('=' * 70)
              print('Processed so far:', files_count, 'out of', input_files_count, '===', files_count / input_files_count, 'good files ratio')
              print('=' * 70)
              count = str(files_count)
              TMIDIX.Tegridy_Any_Pickle_File_Writer(melody_chords_f, '/content/drive/MyDrive/LAKH_INTs_'+count)
              melody_chords_f = []
              print('=' * 70)
        
    except KeyboardInterrupt:
        print('Saving current progress and quitting...')
        break  

    except Exception as ex:
        print('WARNING !!!')
        print('=' * 70)
        print('Bad MIDI:', f)
        print('Error detected:', ex)
        print('=' * 70)
        continue

# Saving last processed files...
print('=' * 70)
print('Saving processed files...')
print('=' * 70)
print('Data check:', min(melody_chords_f), '===', max(melody_chords_f), '===', len(list(set(melody_chords_f))), '===', len(melody_chords_f))
print('=' * 70)
print('Processed so far:', files_count, 'out of', input_files_count, '===', files_count / input_files_count, 'good files ratio')
print('=' * 70)
count = str(files_count)
TMIDIX.Tegridy_Any_Pickle_File_Writer(melody_chords_f, '/content/drive/MyDrive/LAKH_INTs_'+count)

# Displaying resulting processing stats...
print('=' * 70)
print('Done!')   
print('=' * 70)

print('Resulting Stats:')
print('=' * 70)
print('Total good processed MIDI files:', files_count)
print('=' * 70)

print('Instruments stats:')
print('=' * 70)
print('Piano:', stats[0])
print('Guitar:', stats[1])
print('Bass:', stats[2])
print('Violin:', stats[3])
print('Cello:', stats[4])
print('Harp:', stats[5])
print('Trumpet:', stats[6])
print('Clarinet:', stats[7])
print('Flute:', stats[8])
print('Drums:', stats[9])
print('Choir:', stats[10])
print('Organ:', stats[11])
print('=' * 70)

# (TEST INTS)

In [ ]:
#@title Test INTs

train_data1 = melody_chords_f

print('Sample INTs', train_data1[:15])

out = train_data1[:200000]

if len(out) != 0:
    
    song = out
    song_f = []
    time = 0
    dur = 0
    vel = 0
    pitch = 0
    channel = 0

    son = []
    song1 = []

    for s in song:
      if s > 256 and s < (12*9)+640:
        son.append(s)
      else:
        if len(son) == 4:
          song1.append(son)
        son = []
        son.append(s)
                    
    for ss in song1:

      if ss[0] < 128:
        time += ss[0] * 8

      dur = (ss[1]-256) * 16

      if (ss[2]-384) > 128:
        pitch = (ss[2]-384) - 128
      else:
        pitch = (ss[2]-384)


      channel = (ss[3]-640) // 9
      vel = ((ss[3]-640) % 9) * 15
                      
      song_f.append(['note', time, dur, channel, pitch, vel ])

    detailed_stats = TMIDIX.Tegridy_SONG_to_MIDI_Converter(song_f,
                                                        output_signature = 'Orchestrator',  
                                                        output_file_name = '/content/Orchestrator-Music-Composition', 
                                                        track_name='Project Los Angeles',
                                                        list_of_MIDI_patches=[0, 24, 32, 40, 42, 46, 56, 71, 73, 0, 53, 19, 0, 0, 0, 0],
                                                        number_of_ticks_per_quarter=500)

    print('Done!')

# Congrats! You did it! :)